# Pipe Network Analysis
In the pipe network below, the water at 20<sup>o</sup> is following into each manifold at 3000m<sup>3</sup>/h. Each open end of the pipes is connected to a bottom of the tank. The depth of the water in each tank is 15m. Calculate flow in each section of the pipe and calculate the back pressure at the manifolds.

<p style="text-align: center">
    <img src="img/Exercise2_Pipe_Network.jpg" width=600>
</p>

The following information is given for each section of the pipe network. For friction from fittings, an accumulated number of friction coefficient, K, is given for each section. The pipe is clean steel pipe of schedule 40. The height of all the connections including the manifolds and the tank connection is identical.


|   id | Nominal Dia.[mm] | Length[m] | K - fitting |
|-----:|-----------------:|----------:|------------:|
|    1 |              350 |      38.0 |           0 |
|    2 |              350 |      27.0 |         8.0 |
|    3 |              450 |      15.0 |         4.0 |
|    4 |              600 |       5.0 |         7.0 |
|    5 |              350 |      10.0 |        15.0 |
|    6 |              350 |      10.0 |        15.0 |
|    7 |              450 |      24.0 |         6.0 |
|    8 |              350 |      27.0 |         9.0 |
|    9 |              350 |      35.0 |        13.0 |


<p style="text-align: center">
============== Answer ================
</p>

First create a network model and assign the causality to the system.

<p style="text-align: center">
    <img src="causality.png" alt="Insert the image for the network model and causality">
</p>

Write the equations for the system based on the causality.


Let's define a class for a pipe to calculate the pressure drop. You can use this class to calculate the flow from pressure drop or vice versa. Have a look at the class and learn how to use it.

In [8]:
# First we import all the necessary packages and modules
from fluids import friction, core
from fluids.piping import nearest_pipe
from pyfluids import Fluid, FluidsList, Input
import numpy as np
from scipy.optimize import root_scalar, root
import pandas as pd

In [3]:
class Branch:
    """This is a class for a pipe to calculate pressure drop when flow is given or vice versa

    Usage:
        branch = Branch(
            nominal_dia_m=0.03,
            length_m=4,
            schedule="40",
            k_fitting=3.2,
            material="clean steel pipe"
        )
        velocity_m_per_s = branch.get_velocity_m_per_s(volume_flow_m3_per_s=1)
        fluid = Fluid(FluidsList.Water).with_state(Input.pressure(1e5), Input.temperature(20))
        reynolds_number = branch.get_reynolds_number(volume_flow_m3_per_s=1, fluid=fluid)
        pressure_drop = branch.get_pressure_drop_pa(volume_flow_m3_per_s=1, fluid=fluid)
        volume_flow_m3_per_s = branch.get_volume_flow_m3_per_s(pressure_drop_pa=10000, fluid=fluid)
    """

    def __init__(
        self,
        nominal_dia_m: float,
        length_m: float,
        schedule: str,
        k_fitting: float,
        material: str = "clean steel pipe"
    ):
        """This is a constructor. The function is called when you create the class object.

        The arguments should be passed when you create the object
        """
        self.nominal_diameter_m = nominal_dia_m
        self.length_m = length_m
        self.schedule = schedule
        self.material = material
        self.k_fitting = k_fitting
        _, self.inner_dia_m, _, _ = nearest_pipe(Di=self.nominal_diameter_m, schedule=self.schedule)
        self.roughness = friction.material_roughness(self.material)

    def get_section_area_m2(self):
        """Returns area of the pipe section"""
        return self.inner_dia_m**2 * np.pi / 4

    def get_velocity_m_per_s(self, vol_flow_m3_per_s: float) -> float:
        """Returns velocity given the volume flow"""
        return vol_flow_m3_per_s / self.get_section_area_m2()

    def get_reynolds_number(self, vol_flow_m3_per_s: float, fluid: Fluid) -> float:
        """Returns Reynolds number given the volume flow and fluid object"""
        return fluid.density * vol_flow_m3_per_s * self.inner_dia_m / fluid.dynamic_viscosity

    def get_pressure_drop_pa(self, vol_flow_m3_per_s: float, fluid: Fluid) -> float:
        """Returns pressure drop given the volume flow and fluid object

        :param vol_flow_m3_per_s: (float) Volume flow in m3/s
        :param fluid: (Fluid) Fluid object
        :param K: (float) friction coefficient. Default value is 0 if not given.
        """
        pipe_friction = friction.one_phase_dP(
            m=vol_flow_m3_per_s * fluid.density,
            rho=fluid.density,
            mu=fluid.dynamic_viscosity,
            D=self.inner_dia_m,
            roughness=self.roughness,
            L=self.length_m
        )
        fitting_friction = core.dP_from_K(
            K=self.k_fitting,
            rho=fluid.density,
            V=self.get_velocity_m_per_s(vol_flow_m3_per_s)
        )
        return pipe_friction + fitting_friction

    def get_vol_flow_m3_per_s(
            self, pressure_drop_pa: float, fluid: Fluid, initial_vol_flow_m3_per_s: float = None
    ) -> float:
        """Returns the volume flow given the pressure drop and fluid object

        The flow is solved using root_scalar solver.
        """
        if initial_vol_flow_m3_per_s is None:
            initial_velocity = 10000 * fluid.dynamic_viscosity / (fluid.density * self.inner_dia_m)
            initial_vol_flow_m3_per_s = initial_velocity * self.get_section_area_m2()

        def function_to_solve(vol_flow_m3_per_s: float) -> float:
            """Function to solve using the solver"""
            return pressure_drop_pa - self.get_pressure_drop_pa(
                vol_flow_m3_per_s=vol_flow_m3_per_s, fluid=fluid
            )

        sol = root_scalar(f=function_to_solve, x0=initial_velocity, x1=initial_velocity*0.95)
        if not sol.converged:
            raise ValueError("The solution didn't converge")
        return sol.root

(Hint) Use the class above to model the branch in the network model. Have a look at the example in the code to use the methods properly.

```python
branch = Branch(
    nominal_dia_m=0.35,
    length_m=25,
    schedule="40",
    k_fitting=12,
    material="clean steel pipe"
)
```

In [ ]:
schedule = "40"
# Create branch models using the class


Define a function to solve from the system equation written above.

In [34]:
# Boundary conditions
# define variables to be used for boundary condition
# define the fluid to be used in the calculation using Fluid

# define the function to solve. (Hint: as we have multiple pressure nodes in the system, the function should take a vector argument and return the error as a vector
def function_to_solve(p_node_array):
    """Function to solve for the system"""
    # Your code here
    # Return the error vector
    err_array = p_node_array - p_node_array_calculated
    return err_array

In [ ]:
# The solver for the equation requires the initial value. Calculate the initial values for the pressure at the nodes. (Hint: Assume flows at each pipe and calculate the pressure based on the equations. Use your common sense for guessing the initial values)
# your code here for q_branches_initial and p_nodes_initial
# q_branches_initial and p_nodes_initial are numpy arrays

print(f"Initial volume flow [m3/h]:")
for q_init_each in q_branches_initial:
    print(f"{q_init_each * 3600:.0f}")
print(f"\nInitial pressure at nodes [Pa]:")
for p_init_each in p_nodes_initial:
    print(f"{p_init_each:.0f}")

In [36]:
sol = root(fun=function_to_solve, x0=p_nodes_initial)
p_nodes = sol.x

In [37]:
print(f"The pressure at nodes [Pa]:")
for p_node in p_nodes:
    print(f"\t{p_node:.0f}")

The pressure at nodes [Pa]:
	233296
	289499
	333136
	222996


In [38]:
# Calculate the volume flow based on the pressure we acquired
# Your code for calculating q1, q2, ..., q9
vol_flow_m3_per_h_for_branches = np.array([q1, q2, q3, q4, q_fb1, q_fb2, q7, q8, q9]) * 3600
df_pipe_table = pd.DataFrame()
df_pipe_table["vol flow [m3/h]"] = vol_flow_m3_per_h_for_branches
df_pipe_table.index = [index + 1 for index in range(9)]
print(df_pipe_table)

   nominal_dia_m  length_m     K  vol flow [m3/h]
1           0.35      38.0  11.0      1464.199876
2           0.35      27.0   8.0      1720.298138
3           0.45      15.0   4.0      3184.498014
4           0.60       5.0   7.0      6000.000000
5           0.35      10.0  15.0      3000.000000
6           0.35      10.0  15.0      3000.000000
7           0.45      24.0   6.0      2815.501986
8           0.35      27.0   9.0      1531.721049
9           0.35      35.0  13.0      1283.780936


In [39]:
# Calculate the backpressre at the manifolds
p_fb2 = # Your code here
p_fb1 = # Your code here

print("Back pressure at the flow boundaries are [Pa]:")
print(f"\tFB1: {p_fb1:.0f}")
print(f"\tFB2: {p_fb2:.0f}")

Back pressure at the flow boundaries are [Pa]:
	FB1: 747880
	FB2: 747880
